In [ ]:
import torch
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. パラメータ設定 (Parameters)
# ------------------------------------------------------------------------------
# 幾何形状
LENGTH = 100.0  # x: 0 ~ 100
WIDTH = 10.0    # y: 0 ~ 10
HEIGHT = 10.0   # z: 0 ~ 10

# 材料特性 (鋼材)
E = 2.1e5       # ヤング率 (MPa)
nu = 0.3        # ポアソン比

# ラメの定数 (Lame parameters)
mu = E / (2 * (1 + nu))
lmbd = E * nu / ((1 + nu) * (1 - 2 * nu))

# 荷重条件
LOAD_FORCE = 1000.0 # N
AREA = WIDTH * HEIGHT # 100 mm^2
TRACTION_X = LOAD_FORCE / AREA # 10 MPa

# 学習設定
ITERATIONS = 5000 # 状況に応じて増やしてください (例: 15000)
LEARNING_RATE = 1e-3

# ------------------------------------------------------------------------------
# 2. 物理方程式の定義 (PDE & Stress)
# ------------------------------------------------------------------------------
def pde(x, u):
    """
    x: 入力座標 (batch_size, 3) -> [x, y, z]
    u: 変位出力 (batch_size, 3) -> [ux, uy, uz]
    """
    # 変位の勾配 (u_x, u_y, u_z それぞれの x,y,z 微分)
    du_x = dde.grad.jacobian(u, x, i=0, j=0) # du_x/dx
    du_y = dde.grad.jacobian(u, x, i=0, j=1) # du_x/dy
    du_z = dde.grad.jacobian(u, x, i=0, j=2) # du_x/dz
    
    dv_x = dde.grad.jacobian(u, x, i=1, j=0) # du_y/dx
    dv_y = dde.grad.jacobian(u, x, i=1, j=1) # du_y/dy
    dv_z = dde.grad.jacobian(u, x, i=1, j=2) # du_y/dz
    
    dw_x = dde.grad.jacobian(u, x, i=2, j=0) # du_z/dx
    dw_y = dde.grad.jacobian(u, x, i=2, j=1) # du_z/dy
    dw_z = dde.grad.jacobian(u, x, i=2, j=2) # du_z/dz
    
    # ひずみテンソル (Linear Strain)
    E_xx = du_x
    E_yy = dv_y
    E_zz = dw_z
    E_xy = 0.5 * (du_y + dv_x)
    E_yz = 0.5 * (dv_z + dw_y)
    E_zx = 0.5 * (dw_x + du_z)
    
    # 応力テンソル (Hooke's Law)
    trace_E = E_xx + E_yy + E_zz
    S_xx = lmbd * trace_E + 2 * mu * E_xx
    S_yy = lmbd * trace_E + 2 * mu * E_yy
    S_zz = lmbd * trace_E + 2 * mu * E_zz
    S_xy = 2 * mu * E_xy
    S_yz = 2 * mu * E_yz
    S_zx = 2 * mu * E_zx
    
    # 平衡方程式 (Equilibrium Equations) - Body force is ignored
    # d(S_xx)/dx + d(S_xy)/dy + d(S_zx)/dz = 0
    # ...
    
    # 応力の勾配を計算
    S_xx_x = dde.grad.jacobian(S_xx, x, i=0, j=0)
    S_xy_y = dde.grad.jacobian(S_xy, x, i=0, j=1)
    S_zx_z = dde.grad.jacobian(S_zx, x, i=0, j=2)
    
    S_xy_x = dde.grad.jacobian(S_xy, x, i=0, j=0)
    S_yy_y = dde.grad.jacobian(S_yy, x, i=0, j=1)
    S_yz_z = dde.grad.jacobian(S_yz, x, i=0, j=2)
    
    S_zx_x = dde.grad.jacobian(S_zx, x, i=0, j=0)
    S_yz_y = dde.grad.jacobian(S_yz, x, i=0, j=1)
    S_zz_z = dde.grad.jacobian(S_zz, x, i=0, j=2)
    
    momentum_x = S_xx_x + S_xy_y + S_zx_z
    momentum_y = S_xy_x + S_yy_y + S_yz_z
    momentum_z = S_zx_x + S_yz_y + S_zz_z
    
    return [momentum_x, momentum_y, momentum_z]

def compute_stress(x, u):
    """可視化と境界条件適用のため、応力を計算して返すヘルパー関数"""
    du_x = dde.grad.jacobian(u, x, i=0, j=0)
    du_y = dde.grad.jacobian(u, x, i=0, j=1)
    du_z = dde.grad.jacobian(u, x, i=0, j=2)
    dv_x = dde.grad.jacobian(u, x, i=1, j=0)
    dv_y = dde.grad.jacobian(u, x, i=1, j=1)
    dv_z = dde.grad.jacobian(u, x, i=1, j=2)
    dw_x = dde.grad.jacobian(u, x, i=2, j=0)
    dw_y = dde.grad.jacobian(u, x, i=2, j=1)
    dw_z = dde.grad.jacobian(u, x, i=2, j=2)

    E_xx = du_x
    E_yy = dv_y
    E_zz = dw_z
    E_xy = 0.5 * (du_y + dv_x)
    E_yz = 0.5 * (dv_z + dw_y)
    E_zx = 0.5 * (dw_x + du_z)

    trace_E = E_xx + E_yy + E_zz
    sigma_xx = lmbd * trace_E + 2 * mu * E_xx
    sigma_yy = lmbd * trace_E + 2 * mu * E_yy
    sigma_zz = lmbd * trace_E + 2 * mu * E_zz
    sigma_xy = 2 * mu * E_xy
    sigma_yz = 2 * mu * E_yz
    sigma_zx = 2 * mu * E_zx
    
    return sigma_xx, sigma_yy, sigma_zz, sigma_xy, sigma_yz, sigma_zx

# ------------------------------------------------------------------------------
# 3. 幾何形状と境界条件 (Geometry & BCs)
# ------------------------------------------------------------------------------
geom = dde.geometry.Cuboid([0, 0, 0], [LENGTH, WIDTH, HEIGHT])

# 境界の定義
def boundary_fixed(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

def boundary_loaded(x, on_boundary):
    return on_boundary and np.isclose(x[0], LENGTH)

# BC1: 固定端 (x=0) -> 変位ゼロ
bc_fixed_u = dde.DirichletBC(geom, lambda x: 0, boundary_fixed, component=0)
bc_fixed_v = dde.DirichletBC(geom, lambda x: 0, boundary_fixed, component=1)
bc_fixed_w = dde.DirichletBC(geom, lambda x: 0, boundary_fixed, component=2)

# BC2: 載荷端 (x=100) -> Traction applied (Neumann type via OperatorBC)
# n = (1, 0, 0) なので、sigma_xx = 10, sigma_xy = 0, sigma_xz = 0 となるべき
def boundary_loaded_stress_xx(x, y, X):
    # ↓ ここを x (小文字) に変更してください
    s_xx, _, _, _, _, _ = compute_stress(x, y)
    return s_xx - TRACTION_X

def boundary_loaded_stress_xy(x, y, X):
    # ↓ ここを x (小文字) に変更してください
    _, _, _, s_xy, _, _ = compute_stress(x, y)
    return s_xy

def boundary_loaded_stress_xz(x, y, X):
    # ↓ ここを x (小文字) に変更してください
    _, _, _, _, _, s_zx = compute_stress(x, y)
    return s_zx

bc_load_xx = dde.OperatorBC(geom, boundary_loaded_stress_xx, boundary_loaded)
bc_load_xy = dde.OperatorBC(geom, boundary_loaded_stress_xy, boundary_loaded)
bc_load_xz = dde.OperatorBC(geom, boundary_loaded_stress_xz, boundary_loaded)

# 注: 他の面はTraction Freeですが、DeepXDEのPDE損失項が自然境界条件として
# 作用するよう明示的なOperatorBCを追加することも可能ですが、
# 計算コスト削減のため、今回は主要な固定端と荷重端を厳密に拘束します。
# 必要に応じて、y=0, y=10, z=0, z=10 に対する Traction=0 のBCを追加してください。

bcs = [bc_fixed_u, bc_fixed_v, bc_fixed_w, 
       bc_load_xx, bc_load_xy, bc_load_xz]

# ------------------------------------------------------------------------------
# 4. データ設定とモデル構築 (Model Setup)
# ------------------------------------------------------------------------------
# 24^3 = 13824 に近い数を領域点として設定
num_domain = 14000
num_boundary = 3000

data = dde.data.PDE(
    geom,
    pde,
    bcs,
    num_domain=num_domain,
    num_boundary=num_boundary,
    num_test=1000
)

# ネットワーク構成 (入力3 -> [50x4] -> 出力3)
layer_size = [3] + [50] * 4 + [3]
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)

# ------------------------------------------------------------------------------
# 5. 学習 (Training)
# ------------------------------------------------------------------------------
print("Training starts...")
# 座標が大きい(0~100)ため、学習を安定させるためにAdamを使用
model.compile("adam", lr=LEARNING_RATE)
losshistory, train_state = model.train(iterations=ITERATIONS, display_every=500)

# 精度向上のためのL-BFGS (オプション)
# model.compile("L-BFGS")
# losshistory, train_state = model.train()

# ------------------------------------------------------------------------------
# 6. 評価と可視化 (Evaluation & Visualization)
# ------------------------------------------------------------------------------
def get_von_mises(x_coords):
    x_tensor = torch.from_numpy(x_coords).float()
    x_tensor.requires_grad = True
    u_pred = net(x_tensor)
    
    # 応力計算
    # DeepXDEのgraph内計算を再利用できないため、PyTorchで再記述あるいは
    # compute_stressヘルパーを微調整して使う必要がありますが、
    # 簡易的に予測値から再計算します。
    # (ここでは概念実装のため、compute_stressロジックがTensor対応している前提)
    
    # 注意: dde.gradはDeepXDE内部ラッパーなので、外部で使う場合は
    # model.predictの結果を数値微分するか、あるいは PDE内部と同様の計算を行います。
    # ここでは簡易評価として、最大のx変位のみを確認します。
    pass 

print("\n--- Evaluation ---")

# 1. 理論解の計算 (片持ち梁の軸方向変位)
# delta = P * L / (A * E)
P_theory = 1000 # N
A_theory = WIDTH * HEIGHT # 100 mm^2
delta_theory_1000N = (P_theory * LENGTH) / (A_theory * E)
print(f"Theoretical displacement at x=100mm (Load=1000N): {delta_theory_1000N:.5f} mm")

# 2. モデルによる予測 (1000N)
sample_points = np.array([[LENGTH, WIDTH/2, HEIGHT/2]]) # 先端の中心点
u_pred = model.predict(sample_points)
u_x_pred_1000N = u_pred[0, 0]
print(f"Predicted displacement at x=100mm (Load=1000N):   {u_x_pred_1000N:.5f} mm")

# 3. 2000N負荷時の予測 (線形性に基づく外挿)
# 線形弾性解析なので、荷重が2倍になれば変位も2倍になります。
# ユーザー要望: 2000Nの場合の予測 (約0.0095mm程度になるはず)
pred_2000N = u_x_pred_1000N * 2.0
theory_2000N = delta_theory_1000N * 2.0

print(f"\n--- Scenario: Load = 2000 N ---")
print(f"Theoretical Target (approx): 0.00952 mm")
print(f"Linear Extrapolation from PINN: {pred_2000N:.5f} mm")
print(f"Error: {abs(pred_2000N - theory_2000N)/theory_2000N * 100:.2f}%")

# 可視化 (簡単な変形図のプロット)
#  タグは使用できませんが、コードでプロットを生成します。
try:
    import matplotlib.pyplot as plt
    
    # X軸上の点をサンプリング
    x_line = np.linspace(0, LENGTH, 100).reshape(-1, 1)
    y_line = np.full_like(x_line, WIDTH/2)
    z_line = np.full_like(x_line, HEIGHT/2)
    X_line = np.hstack((x_line, y_line, z_line))
    
    u_line = model.predict(X_line)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x_line, u_line[:, 0], label='PINN Prediction ($u_x$)')
    plt.plot(x_line, (x_line / LENGTH) * delta_theory_1000N, 'r--', label='Theoretical ($PL/AE$)')
    plt.title('Displacement $u_x$ along the beam length (Load=1000N)')
    plt.xlabel('x (mm)')
    plt.ylabel('Displacement (mm)')
    plt.legend()
    plt.grid(True)
    plt.show()
    print("Visualization generated.")

except ImportError:
    print("Matplotlib not found, skipping visualization.")

Training starts...
Compiling model...
'compile' took 0.000133 s

Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [3.68e+07, 1.16e+07, 1.74e+07, 2.33e-01, 5.16e-02, 1.13e-01, 1.31e+07, 6.05e+06, 1.62e+05]    [2.49e+07, 7.53e+06, 1.92e+07, 2.33e-01, 5.16e-02, 1.13e-01, 1.31e+07, 6.05e+06, 1.62e+05]    []  
500       [4.61e+03, 5.06e+03, 5.54e+03, 3.13e-04, 7.07e-04, 3.89e-04, 2.74e+01, 4.41e+01, 5.20e+01]    [2.46e+03, 1.96e+03, 3.02e+03, 3.13e-04, 7.07e-04, 3.89e-04, 2.74e+01, 4.41e+01, 5.20e+01]    []  
1000      [1.72e+03, 1.70e+03, 1.96e+03, 1.99e-04, 7.08e-04, 2.67e-04, 1.13e+01, 1.21e+01, 1.91e+01]    [1.06e+03, 6.29e+02, 9.86e+02, 1.99e-04, 7.08e-04, 2.67e-04, 1.13e+01, 1.21e+01, 1.91e+01]    []  
1500      [8.65e+02, 8.66e+02, 1.02e+03, 1.54e-04, 7.09e-04, 2.14e-04, 9.88e+00, 7.59e+00, 1.00e+